# 01 — Data Cleaning: Bruno Fernandes & Casemiro (2025–26)

This notebook loads raw FBref CSV exports for Bruno Fernandes and Casemiro,
cleans them into consistent tables, merges all stat types into a single
per-player master table, and then builds a combined dataset of matches where
both start for Manchester United.

The final output is a single cleaned match-level dataset:

`../data/midfield_matches_clean.csv`

This file will be used by later notebooks for feature engineering, per-90
metrics, visualisations, and radar charts.


In [1]:
import pandas as pd
from pathlib import Path

# Adjust if your CSVs are in a different folder
DATA_DIR = Path("../data")

RAW_FILES = {
    # Bruno
    "Bruno_Summary_Stats": "Bruno_Summary_Stats.csv",
    "Bruno_Miscellanous_Stats": "Bruno_Miscellanous_Stats.csv",
    "Bruno_Possession_Stats": "Bruno_Possession_Stats.csv",
    "Bruno_Passing_Stats": "Bruno_Passing_Stats.csv",
    "Bruno_Pass_Types_Stats": "Bruno_Pass_Types_Stats.csv",
    "Bruno_Goals_Shots_Stats": "Bruno_Goals_Shots_Stats.csv",
    "Bruno_Defensive_Action_Stats": "Bruno_Defensive_Action_Stats.csv",

    # Casemiro
    "Casemiro_Summary_Stats": "Casemiro_Summary_Stats.csv",
    "Casemiro_Miscellaneous_Stats": "Casemiro_Miscellaneous_Stats.csv",
    "Casemiro_Possesion_Stats": "Casemiro_Possesion_Stats.csv",
    "Casemiro_Passing_Stats": "Casemiro_Passing_Stats.csv",
    "Casemiro_Pass_Types_Stats": "Casemiro_Pass_Types_Stats.csv",
    "Casemiro_Goal_Shots_Stats": "Casemiro_Goal_Shots_Stats.csv",
    "Casemiro_Defensive_Actions_Stats": "Casemiro_Defensive_Actions_Stats.csv",
}

OUTPUT_PATH = DATA_DIR / "midfield_matches_clean.csv"

RAW_FILES


{'Bruno_Summary_Stats': 'Bruno_Summary_Stats.csv',
 'Bruno_Miscellanous_Stats': 'Bruno_Miscellanous_Stats.csv',
 'Bruno_Possession_Stats': 'Bruno_Possession_Stats.csv',
 'Bruno_Passing_Stats': 'Bruno_Passing_Stats.csv',
 'Bruno_Pass_Types_Stats': 'Bruno_Pass_Types_Stats.csv',
 'Bruno_Goals_Shots_Stats': 'Bruno_Goals_Shots_Stats.csv',
 'Bruno_Defensive_Action_Stats': 'Bruno_Defensive_Action_Stats.csv',
 'Casemiro_Summary_Stats': 'Casemiro_Summary_Stats.csv',
 'Casemiro_Miscellaneous_Stats': 'Casemiro_Miscellaneous_Stats.csv',
 'Casemiro_Possesion_Stats': 'Casemiro_Possesion_Stats.csv',
 'Casemiro_Passing_Stats': 'Casemiro_Passing_Stats.csv',
 'Casemiro_Pass_Types_Stats': 'Casemiro_Pass_Types_Stats.csv',
 'Casemiro_Goal_Shots_Stats': 'Casemiro_Goal_Shots_Stats.csv',
 'Casemiro_Defensive_Actions_Stats': 'Casemiro_Defensive_Actions_Stats.csv'}

In [2]:
raw = {}
for key, fname in RAW_FILES.items():
    path = DATA_DIR / fname
    df = pd.read_csv(path)
    raw[key] = df

list(raw.keys())


['Bruno_Summary_Stats',
 'Bruno_Miscellanous_Stats',
 'Bruno_Possession_Stats',
 'Bruno_Passing_Stats',
 'Bruno_Pass_Types_Stats',
 'Bruno_Goals_Shots_Stats',
 'Bruno_Defensive_Action_Stats',
 'Casemiro_Summary_Stats',
 'Casemiro_Miscellaneous_Stats',
 'Casemiro_Possesion_Stats',
 'Casemiro_Passing_Stats',
 'Casemiro_Pass_Types_Stats',
 'Casemiro_Goal_Shots_Stats',
 'Casemiro_Defensive_Actions_Stats']

In [3]:
def clean_fbref_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identify the 'Date' header row in an FBref export, use it as the header,
    and return the cleaned data portion.
    """
    first_col = df.columns[0]
    header_row_idx = None

    for idx, val in df[first_col].items():
        if isinstance(val, str) and val.strip() == "Date":
            header_row_idx = idx
            break

    if header_row_idx is None:
        raise ValueError("No 'Date' header row found")

    header = df.iloc[header_row_idx]
    data = df.iloc[header_row_idx + 1 :].reset_index(drop=True).copy()
    data.columns = header

    # Drop all-NaN columns and strip whitespace from column names
    data = data.dropna(axis=1, how="all")
    data.columns = [str(c).strip() for c in data.columns]

    return data


In [4]:
cleaned = {name: clean_fbref_table(df) for name, df in raw.items()}

cleaned["Bruno_Summary_Stats"].head()


,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,GCA,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ,Match Report
0,8/17/25,Sun,Premier League,Matchweek 1,Home,L 0–1,Manchester Utd,Arsenal,Y*,CM,...,0,49,62,79,10,35,2,2,1,Match Report
1,8/24/25,Sun,Premier League,Matchweek 2,Away,D 1–1,Manchester Utd,Fulham,Y*,CM,...,0,47,57,82.5,5,37,1,3,2,Match Report
2,8/27/25,Wed,EFL Cup,Second round,Away,D 2 (11)–2 (12),Manchester Utd,Grimsby Town,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report
3,8/30/25,Sat,Premier League,Matchweek 3,Home,W 3–2,Manchester Utd,Burnley,Y*,"CM,AM",...,0,73,90,81.1,7,60,3,0,0,Match Report
4,9/6/25,Sat,WCQ,First round,Away,W 5–0,pt Portugal,am Armenia,Y,CM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report


In [5]:
JOIN_KEYS = [
    "Date", "Day", "Comp", "Round", "Venue",
    "Result", "Squad", "Opponent", "Start", "Pos", "Min",
]

def make_master(player_prefix: str) -> pd.DataFrame:
    """
    Build a single wide table for one player by merging:
    - Summary
    - Misc
    - Possession
    - Passing
    - Pass types
    - Goals/Shots
    - Defensive actions

    If a metric column (e.g. 'Bruno_Match Report') already exists
    in master, it is dropped from new tables to avoid duplicates.
    """
    if player_prefix == "Bruno":
        suffixes = [
            "Summary_Stats",
            "Miscellanous_Stats",
            "Possession_Stats",
            "Passing_Stats",
            "Pass_Types_Stats",
            "Goals_Shots_Stats",
            "Defensive_Action_Stats",
        ]
    else:  # Casemiro
        suffixes = [
            "Summary_Stats",
            "Miscellaneous_Stats",
            "Possesion_Stats",
            "Passing_Stats",
            "Pass_Types_Stats",
            "Goal_Shots_Stats",
            "Defensive_Actions_Stats",
        ]

    master = cleaned[f"{player_prefix}_Summary_Stats"].copy()

    for suf in suffixes[1:]:
        name = f"{player_prefix}_{suf}"
        df = cleaned[name].copy()

        metric_cols = [c for c in df.columns if c not in JOIN_KEYS]
        rename_map = {c: f"{player_prefix}_{c}" for c in metric_cols}

        df_renamed = df[JOIN_KEYS + metric_cols].rename(columns=rename_map)

        metric_prefixed_cols = [rename_map[c] for c in metric_cols]
        new_metric_cols = [c for c in metric_prefixed_cols if c not in master.columns]

        if not new_metric_cols:
            continue

        df_renamed = df_renamed[JOIN_KEYS + new_metric_cols]
        master = master.merge(df_renamed, on=JOIN_KEYS, how="left")

    return master


In [6]:
bruno_master = make_master("Bruno")
casemiro_master = make_master("Casemiro")

bruno_master.shape, casemiro_master.shape


((17, 276), (100, 276))

In [7]:
def is_start_flag(val) -> bool:
    # 'Y' or 'Y*' are usual start indicators
    return isinstance(val, str) and "Y" in val

bruno_mu_start = bruno_master[
    (bruno_master["Squad"] == "Manchester Utd") &
    (bruno_master["Start"].apply(is_start_flag))
].copy()

casemiro_mu_start = casemiro_master[
    (casemiro_master["Squad"] == "Manchester Utd") &
    (casemiro_master["Start"].apply(is_start_flag))
].copy()

bruno_mu_start.shape, casemiro_mu_start.shape


((11, 276), (9, 276))

In [8]:
MATCH_KEYS = ["Date", "Comp", "Round", "Venue", "Result", "Squad", "Opponent"]

both_start_raw = bruno_mu_start.merge(
    casemiro_mu_start,
    on=MATCH_KEYS,
    how="inner",
    suffixes=("_BrunoRow", "_CasemiroRow"),
)

both_start_raw.shape


(9, 545)

In [9]:
# Minutes from per-player tables
bruno_min = bruno_mu_start[MATCH_KEYS + ["Min"]].rename(columns={"Min": "Bruno_Min"})
casemiro_min = casemiro_mu_start[MATCH_KEYS + ["Min"]].rename(columns={"Min": "Casemiro_Min"})

minutes_df = bruno_min.merge(casemiro_min, on=MATCH_KEYS, how="inner")

# Merge minutes onto both_start_raw
both_start = both_start_raw.merge(minutes_df, on=MATCH_KEYS, how="left")

[b for b in both_start.columns if "Min" in b]


['Min_BrunoRow', 'Min_CasemiroRow', 'Bruno_Min', 'Casemiro_Min']

In [11]:
both_start = both_start.drop(columns=[
    "Min_BrunoRow", 
    "Min_CasemiroRow"
], errors="ignore")

[b for b in both_start.columns if "Min" in b]


['Bruno_Min', 'Casemiro_Min']

In [12]:
meta_cols = MATCH_KEYS.copy()
extra_meta = []  # if you want to add e.g. 'Day' later

bruno_metric_cols = [c for c in both_start.columns if c.startswith("Bruno_")]
casemiro_metric_cols = [c for c in both_start.columns if c.startswith("Casemiro_")]

cols_order = meta_cols + ["Bruno_Min", "Casemiro_Min"] + bruno_metric_cols + casemiro_metric_cols

# Only keep columns that actually exist
cols_order = [c for c in cols_order if c in both_start.columns]

both_start_clean = both_start[cols_order].copy()
both_start_clean.shape


(9, 17289)

In [13]:
both_start_clean.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH, both_start_clean.shape


(PosixPath('../data/midfield_matches_clean.csv'), (9, 17289))